In [1]:
import numpy as np
from qiskit_experiments.library import StandardRB, InterleavedRB
from qiskit_experiments.framework import ParallelExperiment, BatchExperiment
import qiskit.circuit.library as circuits
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakePerth
from qiskit import transpile
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qutip_qip.qiskit import QiskitCircuitSimulator,QiskitPulseSimulator
import numpy as np
from typing import List,Optional,Union ,Dict, Any
import matplotlib.pyplot as plt
import qutip
from qutip import basis, fidelity,sigmax, sigmay, sigmaz,identity,rand_unitary,tensor,mesolve,Qobj, Bloch
from qutip_qip.operations import  rx,ry,rz,cz_gate,expand_operator
from typing import List
from tqdm import tqdm
from qutip_qip.device import Processor, Model
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
import qiskit
import numpy as np
import copy
import uuid
import random
from collections import Counter
import qutip
import qiskit
from qutip import basis
from qutip_qip.circuit import QubitCircuit
from qutip_qip.circuit.circuitsimulator import CircuitResult
from qutip_qip.device import Processor
from qutip_qip.qiskit import QiskitSimulatorBase
from qutip_qip.qiskit.job import Job
from qutip_qip.qiskit.converter import convert_qiskit_circuit
from qiskit.providers import BackendV1,BackendV2, Options
from qiskit.providers.models import QasmBackendConfiguration
from qiskit.result import Result, Counts
from qiskit.result.models import ExperimentResult, ExperimentResultData
from qiskit.quantum_info import Statevector, DensityMatrix
from qiskit.circuit import QuantumCircuit
from qiskit.qobj import QobjExperimentHeader
from qiskit.transpiler import Target
from qiskit.circuit.library import RXGate, RZGate, CZGate
from qiskit.circuit import Parameter
from qiskit.providers.provider import ProviderV1
from qiskit.circuit.instruction import Instruction

/var/folders/9k/z8bpg3hj3d9841ggs75xmwxw0000gn/T/ipykernel_70137/3492694123.py:38: DeprecationWarning: qiskit.providers.models is deprecated since Qiskit 1.2 and will be removed in Qiskit 2.0. With the removal of Qobj, there is no need for these schema-conformant objects. If you still need to use them, it could be because you are using a BackendV1, which is also deprecated in favor of BackendV2.
  from qiskit.providers.models import QasmBackendConfiguration


In [2]:
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)
class MyProvider(ProviderV1):
    def __init__(self):
        super().__init__()

    def backends(self, name=None, **kwargs):
        # Return a list of available backends
        # For simplicity, return an empty list or a list with your custom backend
        return [MyQiskitPulseSimulator()] if name is None else []

class MyBackendV2(BackendV2):
    def __init__(self):
        self.shots = 1024
    def run(self, qiskit_circuit: QuantumCircuit, **run_options) -> Job:
        print('MyBackendV2.run called')
        # self.set_options(
        #     shots=(
        #         run_options["shots"]
        #         if "shots" in run_options
        #         else self._default_options().shots
        #     ),
        #     allow_custom_gate=(
        #         run_options["allow_custom_gate"]
        #         if "allow_custom_gate" in run_options
        #         else self._default_options().allow_custom_gate
        #     ),
        # )
        
        job_id = str(uuid.uuid4())
        print('MyBackendV2 job_id generated')
        job = Job(
            backend=self,
            job_id=job_id,
            result=self._run_job(job_id, qiskit_circuit),
        )
        return job

    def _sample_shots(self, count_probs: dict) -> Counts:
        shots = self.shots
        samples = random.choices(
            list(count_probs.keys()), list(count_probs.values()), k=shots
        )
        return Counts(Counter(samples))

    def _get_probabilities(self, state):
        if state.type == "oper":
            return state.diag()
        return np.array([np.abs(coef) ** 2 for coef in state])

class MyQiskitPulseSimulator(MyBackendV2):
    def __init__(self, processor: Processor, configuration=None, **fields):
        self._provider = MyProvider() 
        self.processor = processor
        self.name = 'fake NA backend'
        self.version = '0.1'
        super().__init__( **fields)

        target = Target(description='fake NA backend')
        target.add_instruction(CZGate(),{(0,1):None})
        theta = Parameter('theta')
        target.add_instruction(
            instruction= Instruction(name = 'rx', 
                                           params = [theta],
                                           num_qubits = 1,
                                           num_clbits=0),
            properties = {(0,):None,
                        (1,):None},
            name = 'rx')
        
        target.add_instruction(
            instruction= Instruction(name = 'rz', 
                                           params = [theta],
                                           num_qubits = 1,
                                           num_clbits=0),
            properties = {(0,):None,
                        (1,):None},
            name = 'rz')

        target.add_instruction(
            instruction= Instruction(name = 'reset', 
                                           params = [],
                                           num_qubits = 1,
                                           num_clbits=0),
            properties = {(0,):None,
                        (1,):None},
            name = 'reset')
        
        target.add_instruction(
            instruction= Instruction(name = 'measure', 
                                           params = [],
                                           num_qubits = 1,
                                           num_clbits=0),
            properties = {(0,):None,
                        (1,):None},
            name = 'measure')
        
        self._target = target
                        
    def _parse_results(
        self, final_state: qutip.Qobj, job_id: str, qutip_circuit: QubitCircuit
    ) -> qiskit.result.Result:
        count_probs = {}
        counts = None

        # calculate probabilities of required states
        if final_state:
            for i, prob in enumerate(self._get_probabilities(final_state)):
                if not np.isclose(prob, 0):
                    count_probs[hex(i)] = prob
            # sample the shots from obtained probabilities
            counts = self._sample_shots(count_probs)

        exp_res_data = ExperimentResultData(
            counts=counts,
            statevector=(
                Statevector(data=final_state.full())
                if final_state.type == "ket"
                else DensityMatrix(data=final_state.full())
            ),
        )

        header = QobjExperimentHeader.from_dict(
            {
                "name": (
                    qutip_circuit.name
                    if hasattr(qutip_circuit, "name")
                    else ""
                ),
                "n_qubits": qutip_circuit.N,
            }
        )

        exp_res = ExperimentResult(
            shots=self.shots,
            success=True,
            data=exp_res_data,
            header=header,
        )

        result = Result(
            backend_name='fake NA backend',
            backend_version='0.1',
            qobj_id=id(qutip_circuit),
            job_id=job_id,
            success=True,
            results=[exp_res],
        )

        return result

    def _run_job(self, job_id: str, 
                #  qutip_circuit: QubitCircuit
                qiskit_circuit: QuantumCircuit
                 ) -> Result:
        print('_run_job called')
        zero_state = self.processor.generate_init_processor_state()

        # self.processor.load_circuit(qutip_circuit)
        self.processor.load_circuit(qiskit_circuit)
        result = self.processor.run_state(zero_state)

        final_state = self.processor.get_final_circuit_state(result.states[-1])

        qutip_circ = convert_qiskit_circuit(
            qiskit_circuit,
            allow_custom_gate=self.options.allow_custom_gate,
        )

        return self._parse_results(
            final_state=final_state, job_id=job_id, qutip_circuit=qutip_circ
        )

    @classmethod
    def _default_options(cls):
        return Options(shots=1024, allow_custom_gate=True)
    
    @property
    def max_circuits(self):
        return 1000
    
    @property
    def target(self):
        return self._target
    
GROUND = basis(4, 0)
EXCITED = basis(4, 1)
LEAKAGE = basis(4, 2)
RYDBERG = basis(4, 3)

class minimal_processor(Processor):
    def __init__(self,
                 num_qubits=2,
                 Omega_01 = 9.999385781382449,
                 delta_1 = -9.997619824080692,
                 gamma_r = 0,# 1 / 540
                 ):
        self.Omega_01 = Omega_01
        self.delta_1= delta_1
        self.basis_gates=['rx','rz','cz']
        self.qbt_dim=4
        self.model=Model(num_qubits=num_qubits,
                         dims=[self.qbt_dim for _ in range(num_qubits)])
        self.num_qubits=num_qubits
        self.get_c_ops(gamma_r)

    def evolve(self, state: qutip.Qobj, name: str, param: Optional[float], qbts: List[int]):
        if name == 'rx':
            return self._evolve_rx(state, param, qbts)
        elif name == 'rz':
            return self._evolve_rz(state, param, qbts)
        elif name == 'cz':
            return self._evolve_cz(state, qbts)
        # else:
        #     raise ValueError(f"Unsupported gate: {name}")

    def _evolve_rx(self, state: qutip.Qobj, param: float, qbts: List[int]):
        Omega_01 = self.Omega_01
        return mesolve(
            H=self.id_wrap(Omega_01 / 2 * (GROUND * EXCITED.dag() + EXCITED * GROUND.dag()), qbts[0]),
            rho0=state,
            tlist=np.array([0, param / 10]),
            c_ops=self.collapse_ops,
            options={'store_final_state': True}
        )

    def _evolve_rz(self, state: qutip.Qobj, param: float, qbts: List[int]):
        delta_1 = self.delta_1
        H = self.id_wrap(delta_1 * EXCITED * EXCITED.dag(), qbts[0])
        return mesolve(
            H=H,
            rho0=state,
            tlist=np.array([0, param / 10]),
            c_ops=self.collapse_ops,
            options={'store_final_state': True}
        )

    def _evolve_cz(self, state: qutip.Qobj, qbts: List[int]):
        t_tot = 0.540  # microsecond, which is the total duration of the gate protocol
        B = 200 * 2 * np.pi  # interaction strength
        omegaMax = 17 * 2 * np.pi  # MHz
        deltaMax = 23 * 2 * np.pi  # MHz
        tau = 0.175 * t_tot
        a = np.exp(-(t_tot / 4) ** 4 / tau ** 4)

        def Rabi_frequency(t, args):
            return (t < t_tot / 2) * omegaMax * (np.exp(-(t - t_tot / 4) ** 4 / tau ** 4) - a) / (1 - a) + \
                   (t >= t_tot / 2) * omegaMax * (np.exp(-(t - 3 * t_tot / 4) ** 4 / tau ** 4) - a) / (1 - a)

        def Detunning(t, args):
            return (t < t_tot / 2) * (-1) * deltaMax * np.cos(2 * np.pi * t / t_tot) + \
                   (t >= t_tot / 2) * deltaMax * np.cos(2 * np.pi * t / t_tot)

        H = []
        for idx in qbts:
            H.append([self.id_wrap(0.5 * RYDBERG * EXCITED.dag() + 0.5 * EXCITED * RYDBERG.dag(), idx), Rabi_frequency])
            H.append([self.id_wrap(RYDBERG * RYDBERG.dag(), idx), Detunning])

        H.append(self.id_wrap(B * tensor(RYDBERG, RYDBERG) * tensor(RYDBERG, RYDBERG).dag(), qbts))  # This assumes the system involves two qubit only.

        state = self._evolve_rz(state, np.pi, [qbts[0]]).states[-1]
        state = mesolve(
            H=H,
            rho0=state,
            tlist=np.array([0, t_tot]),
            c_ops=self.collapse_ops,
            options={'store_final_state': True, 'nsteps': 10000}
        ).states[-1]
        return self._evolve_rz(state, np.pi, [qbts[1]])

    def invert_idx(self,idx):
        return self.num_qubits-1-idx        
    
    def run_state(self,
                  init_state:qutip.Qobj) -> qutip.solver.Result:
        print('run_state called')
        if init_state.isket:
            assert init_state.dims == [[self.qbt_dim for _ in range(self.num_qubits)],[1 for _ in range(self.num_qubits)]]
        else:
            assert init_state.dims == [[self.qbt_dim for _ in range(self.num_qubits)],[self.qbt_dim for _ in range(self.num_qubits)]]
        state = copy.deepcopy(init_state)
        if isinstance(self.qiskit_circ_transpiled,QuantumCircuit):
            for ins in tqdm(self.qiskit_circ_transpiled.data,'Simulating gate'):
                print(ins)
                name = ins.operation.name    
                param = next(iter(ins.operation.params), None)
                qbts = [self.invert_idx(qubit._index) for qubit in ins.qubits]
                result = self.evolve(state, name, param, qbts)            
                state = result.final_state
            return result
        else:
            results = []
            for circ in tqdm(self.qiskit_circ_transpiled,'Simulating circuits') :
                for ins in circ.data:
                    name = ins.operation.name    
                    param = next(iter(ins.operation.params), None)
                    qbts = [self.invert_idx(qubit._index) for qubit in ins.qubits]
                    if name in self.basis_gates:
                        result = self.evolve(state, name, param, qbts)            
                        state = result.final_state
                results.append(result)
            return results
    def generate_init_processor_state(self)->qutip.Qobj:
        # Always initialize in zero
        return tensor([basis(self.qbt_dim, 0) for _ in range(self.num_qubits)])
    
    def id_wrap(self,
                op:qutip.Qobj,
                idx:Union[int,List[int]])->qutip.Qobj:
        if isinstance(idx,int):
            idx = [idx]
        return expand_operator(oper=op,
                               dims=[self.qbt_dim for _ in range(self.num_qubits)],
                               targets=idx)

    def get_c_ops(self,gamma_r):
        L0 = np.sqrt(1/16 * gamma_r) * (GROUND * RYDBERG.dag())
        L1 = np.sqrt(1/16 * gamma_r) * (EXCITED * RYDBERG.dag())
        Ld = np.sqrt(7/8 * gamma_r) * (LEAKAGE * RYDBERG.dag())
        single_q_c_ops = [L0, L1, Ld]

        self.collapse_ops = []
        for q in range(self.num_qubits):
            for c_op in single_q_c_ops:
                self.collapse_ops.append(self.id_wrap(c_op,
                                                      q))
    
    def load_circuit(self,
                     qiskit_circuit:qiskit.QuantumCircuit)->None:
        self.qiskit_circ_transpiled = transpile(qiskit_circuit,basis_gates=self.basis_gates)
        # self.qiskit_circ_transpiled = qiskit_circ_transpiled
    
    def get_final_circuit_state(self,state:qutip.Qobj)->qutip.Qobj:
        rho = state
        if rho.isket:
            rho = qutip.ket2dm(rho)
        rho_arr = rho.full()
        rho_reshaped = rho_arr.reshape(*[self.qbt_dim for _ in range(self.num_qubits*2)])
        # print(rho_reshaped.shape)
        rho_reshaped_2lvl = rho_reshaped[tuple(slice(0, 2) for _ in range(2 * self.num_qubits))]
        # print(rho_reshaped_2lvl.shape)
        return Qobj(rho_reshaped_2lvl.reshape(2**self.num_qubits , 2**self.num_qubits), dims=[[2] * self.num_qubits, [2] * self.num_qubits])
    

In [3]:
processor = minimal_processor(2,gamma_r=0)
pulse_backend = MyQiskitPulseSimulator(processor)


lengths = np.arange(1, 800, 200)
num_samples = 10
seed = 1010
qubits = [0]

# Run an RB experiment on qubit 0
experiment = StandardRB(qubits, lengths, num_samples=num_samples, seed=seed)
experiment.set_transpile_options(basis_gates=['rx','rz','cz','reset','measure'])


experiment._set_backend(pulse_backend)
experiment._finalize()
transpiled_circuits = experiment._transpiled_circuits()
experiment_data = experiment._initialize_experiment_data()
run_opts = experiment.run_options.__dict__
jobs = experiment._run_jobs(transpiled_circuits, sampler=None, **run_opts)

/var/folders/9k/z8bpg3hj3d9841ggs75xmwxw0000gn/T/ipykernel_70137/109590167.py:5: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  super().__init__()


MyBackendV2.run called
MyBackendV2 job_id generated
_run_job called


/Users/jiakaiwang/Documents/GitHub/physics707/.venv/lib/python3.13/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:233: UserWarning: Options {'execution': {'meas_type': 'classified'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


Simulating circuits: 100%|██████████| 40/40 [00:21<00:00,  1.89it/s]


In [4]:
for item in transpiled_circuits[2].data:
    print(item)

CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-1.5707963267948966]), qubits=(Qubit(QuantumRegister(1, 'q'), 0),), clbits=())
CircuitInstruction(operation=Instruction(name='rx', num_qubits=1, num_clbits=0, params=[3.141592653589793]), qubits=(Qubit(QuantumRegister(1, 'q'), 0),), clbits=())
CircuitInstruction(operation=Instruction(name='barrier', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(1, 'q'), 0),), clbits=())
CircuitInstruction(operation=Instruction(name='rx', num_qubits=1, num_clbits=0, params=[-1.5707963267948966]), qubits=(Qubit(QuantumRegister(1, 'q'), 0),), clbits=())
CircuitInstruction(operation=Instruction(name='barrier', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(1, 'q'), 0),), clbits=())
CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[-1.5707963267948966]), qubits=(Qubit(QuantumRegister(1, 'q'), 0),), clbits=())
CircuitInstruction(operation

run_state called


## To determine what providors.Job should look like, I should refer to ExperimentData.add_jobs?

# Another thing I haven't implemented is how measurement is handled. Measurement probably causes the job results to be invalid

In [2]:
import qutip
qutip.__version__

'5.0.4'

In [ ]:
# Run an RB experiment on qubit 0
exp1 = StandardRB(qubits, lengths, num_samples=num_samples, seed=seed)
expdata1 = exp1.run(pulse_backend).block_for_results()
results1 = expdata1.analysis_results()

# View result data
print("Gate error ratio: %s" % expdata1.experiment.analysis.options.gate_error_ratio)